In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,Dataset

In [ ]:
class CustomImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        image, label = super().__getitem__(index)
        
        label = torch.tensor(label, dtype=torch.float)
        
        return image, label

trans=transforms.Compose([
          transforms.Resize((64,64)),

          transforms.ToTensor(),
          transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
]
)
root_dir=r'C:\Users\navee\Desktop\flask_apps\dog_cat_classification\dataset\train'
dataset = CustomImageFolder(root=root_dir, transform=trans)

In [7]:
inp_siz=3
epoch=200
batch=100
lr=0.00001
dataload=DataLoader(dataset,batch_size=batch,shuffle=True)



In [4]:
class model_arc(nn.Module):
          def __init__(self,inp_siz):
                  super(model_arc,self).__init__()
                  self.con1=nn.Conv2d(in_channels=inp_siz,out_channels=16,kernel_size=3,stride=1,padding=1)
                  self.b1=nn.BatchNorm2d(16) 

                  self.con2=nn.Conv2d(16,out_channels=32,kernel_size=5,stride=1,padding=1)
                  self.b2=nn.BatchNorm2d(32) 
                  
                  self.con3=nn.Conv2d(32,out_channels=64,kernel_size=3,stride=1,padding=1)
                  self.b3=nn.BatchNorm2d(64) 

                  self.con4=nn.Conv2d(64,out_channels=128,kernel_size=3,stride=1,padding=1)
                  self.b4=nn.BatchNorm2d(128)

                  self.maxp=nn.MaxPool2d(kernel_size=4,stride=2) 
                  self.relu=nn.ReLU()
                  self.d1=nn.Dropout2d(p=0.15)
                  self.d2=nn.Dropout2d(p=0.25)
                  self.d3=nn.Dropout2d(p=0.5)
                  self.d4=nn.Dropout(p=0.5)

                  self.l1=nn.Linear(128*14*14,512)
                  self.l2=nn.Linear(512,1)
          def forward(self,inp):
                  x=self.d1(self.relu(self.b1(self.con1(inp)))) # 400,16,64,64
                  x=self.d2(self.relu((self.b2(self.con2(x))))) # 400,32,62,62
                  x=self.d3(self.relu((self.b3(self.con3(x))))) # 400,64,62,62
                  x=self.maxp(x)  # 400,64,30,30
                  x=self.relu(self.b4(self.con4(x)))
                  x=self.maxp(x)  # 400,128,14,14

                  x=self.d4(self.relu(self.l1(torch.flatten(x,1))))
                  out=self.l2(x)

                  return out

In [8]:
model=model_arc(inp_siz).to('cuda')
loss=nn.BCEWithLogitsLoss().to('cuda')
opt=torch.optim.Adam(model.parameters(),lr)

for i in range(epoch):
          for inp,out in dataload:
                  inp=inp.to('cuda')
                  out=out.to('cuda')
                  out=out.view(-1,1)
                  opt.zero_grad()
                  y=model.forward(inp)
                  l=loss(y,out)
                  l.backward()
                  opt.step()
          if (i+1)%10==0:
                  print(f"{i+1}/{epoch} loss is {l}")

          

10/200 loss is 0.6371074318885803
20/200 loss is 0.6334914565086365
30/200 loss is 0.6434234380722046
40/200 loss is 0.5887530446052551
50/200 loss is 0.657886803150177
60/200 loss is 0.6163897514343262
70/200 loss is 0.5469484329223633
80/200 loss is 0.4964214265346527
90/200 loss is 0.4902949929237366
100/200 loss is 0.455278605222702
110/200 loss is 0.47908490896224976
120/200 loss is 0.4879530370235443
130/200 loss is 0.4384113848209381
140/200 loss is 0.545909583568573
150/200 loss is 0.5390331745147705
160/200 loss is 0.40672609210014343
170/200 loss is 0.43863385915756226
180/200 loss is 0.3020067811012268
190/200 loss is 0.44496870040893555
200/200 loss is 0.3521955609321594


In [9]:
# Save the entire model (less flexible)
torch.save(model, 'entire_model-2.pth')
torch.save(model.state_dict(), 'model_weights.pth')
